In [175]:
%%capture
%pip install fuzzywuzzy openpyxl pandas polars numpy
%pip install git+https://github.com/seandavi/cu_profile_utils

In [176]:
import pandas as pd
import polars as pl

In [177]:
x = pl.read_excel('/Users/seandavis/Downloads/Active Full-Mentored-Affiliate-Clinical Members_12.8.22.xlsx')
y = pl.read_parquet('/Users/seandavis/Documents/git/infra/dagster-infra/cu_profiles.parquet')
z = pl.read_excel('/Users/seandavis/Downloads/2023pubdataset_all.xlsx')

In [178]:
x.head()

Member ID,Last Name,First Name,Middle Initial,Program,Type,Status,Status Date,Institution,School,Dept,Div,Email,ORCID
i64,str,str,str,str,str,str,str,str,str,str,str,str,str
2004,"""Adgate""","""John""",null,"""CPC""","""Full""","""Active""","""10-Apr-20""","""UCD""","""School of Publ...","""Environmental ...",null,"""john.adgate@cu...",null
1392,"""Agarwal""","""Chapla""",null,"""CPC""","""Affiliate""","""Active""","""29-Nov-03""","""UCD""","""Skaggs School ...","""Pharmaceutical...",null,"""chapla.agarwal...",null
1100,"""Agarwal""","""Rajesh""",null,"""CPC""","""Full""","""Active""","""01-Sep-98""","""UCD""","""Skaggs School ...","""Pharmaceutical...",null,"""rajesh.agarwal...","""Orcid: 0000-00..."
1260,"""Ahn""","""Natalie""","""G""","""MCO""","""Full""","""Active""","""01-Aug-94""","""CU-Boulder""","""College of Art...","""Chemistry & Bi...",null,"""Natalie.Ahn@co...","""Orcid: 0000-00..."
2091,"""Ahrendt""","""Gretchen""","""McGuire""","""CPC""","""Clinical""","""Active""","""09-Sep-22""","""UCD""","""School of Medi...","""Surgery""","""Surgical Oncol...","""gretchen.ahren...",null


In [179]:
y.head()

name,institution,department,title,profile_id,__index_level_0__
str,str,str,str,i64,i64
"""Laurra Michell...","""University of ...","""CON-DEAN Admin...","""Instructor""",12755836,0
"""Jenny A. Aalbo...","""University of ...","""CSPH-LEAD GEN ...","""Instructor""",31093281,1
"""Alec Aaronson""","""University of ...","""SOM-PSYCH""","""Instructor""",40681884,2
"""Michael David ...","""University of ...","""SOM-PSYCH""","""Instructor""",231137,3
"""Mona M Abaza""","""University of ...","""SOM-OTO Genera...","""Professor""",225586,4


In [180]:
print(z
      .groupby('Members')
      .count()
      .sort('count', descending=True)
      .filter(pl.col('Members').str.contains('Davis'))
)

shape: (2, 2)
┌─────────────────────────────────────┬───────┐
│ Members                             ┆ count │
│ ---                                 ┆ ---   │
│ str                                 ┆ u32   │
╞═════════════════════════════════════╪═══════╡
│ DavisSL-1805                        ┆ 2     │
│ DavisSL-1805, Diamond-1673, Lieu... ┆ 1     │
└─────────────────────────────────────┴───────┘


In [181]:
z.filter(pl.col('Members').str.contains('Davis'))

PMID,PMCID,Journal,Citation,PubYear,Members,Programs,Coding_Report,Coding_InterIntraCollaboration,SRUse,MemberInst,SpecialFocus
i64,str,str,str,i64,str,str,str,str,str,str,str
36152015,null,"""Hepatology""","""Borad MJ, Bai ...",2022,"""DavisSL-1805""","""CPC""","""01-Report""","""0000, 0004, 04...","""None""","""9""","""None"""
36309653,"""PMC9617348""","""BMC Cancer""","""Davis SL (CPC)...",2022,"""DavisSL-1805, ...","""CPC, DT""","""01-Report""","""0003, 0402, 05...","""BDC-MolBio, FC...","""9""","""None"""
36417147,null,"""Curr Treat Opt...","""Lenneman C, Ha...",2022,"""DavisSL-1805""","""CPC""","""02-Review""","""0000, 0400""","""None""","""9""","""None"""


In [183]:
y1 = y.with_columns(
    pl.col('name').str.split(' ').alias('list_names')
).with_columns(
    pl.col('list_names').arr.first().alias('first_name'),
    pl.col('list_names').arr.last().alias('last_name')
)# .join(x, left_on=['first_name','last_name'], right_on=['First Name', 'Last Name'], type='right')
y2 = x.join(y1, right_on=['first_name','last_name'], left_on=['First Name', 'Last Name'], how='left')
y2.head()

Member ID,Last Name,First Name,Middle Initial,Program,Type,Status,Status Date,Institution,School,Dept,Div,Email,ORCID,name,institution,department,title,profile_id,__index_level_0__,list_names
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str,i64,i64,list[str]
2004,"""Adgate""","""John""",null,"""CPC""","""Full""","""Active""","""10-Apr-20""","""UCD""","""School of Publ...","""Environmental ...",null,"""john.adgate@cu...",null,"""John Lawrence ...","""University of ...","""CSPH-EOH Gener...","""Professor""",226192,62,"[""John"", ""Lawrence"", ""Adgate""]"
1392,"""Agarwal""","""Chapla""",null,"""CPC""","""Affiliate""","""Active""","""29-Nov-03""","""UCD""","""Skaggs School ...","""Pharmaceutical...",null,"""chapla.agarwal...",null,"""Chapla Agarwal...","""University of ...","""SOP-Administra...","""Professor""",226697,70,"[""Chapla"", ""Agarwal""]"
1100,"""Agarwal""","""Rajesh""",null,"""CPC""","""Full""","""Active""","""01-Sep-98""","""UCD""","""Skaggs School ...","""Pharmaceutical...",null,"""rajesh.agarwal...","""Orcid: 0000-00...","""Rajesh Agarwal...","""University of ...","""SOP-Administra...","""Professor""",226974,71,"[""Rajesh"", ""Agarwal""]"
1260,"""Ahn""","""Natalie""","""G""","""MCO""","""Full""","""Active""","""01-Aug-94""","""CU-Boulder""","""College of Art...","""Chemistry & Bi...",null,"""Natalie.Ahn@co...","""Orcid: 0000-00...","""Natalie Ahn""","""University of ...","""Biochemistry""","""Professor""",224616,83,"[""Natalie"", ""Ahn""]"
2091,"""Ahrendt""","""Gretchen""","""McGuire""","""CPC""","""Clinical""","""Active""","""09-Sep-22""","""UCD""","""School of Medi...","""Surgery""","""Surgical Oncol...","""gretchen.ahren...",null,"""Gretchen Ahren...","""University of ...","""SOM-SRG""","""Professor""",14111566,85,"[""Gretchen"", ""Ahrendt""]"


In [184]:
scimago = pl.read_csv('https://www.scimagojr.com/journalrank.php?out=xls', sep=";")
scimago.head()

Rank,Sourceid,Title,Type,Issn,SJR,SJR Best Quartile,H index,Total Docs. (2022),Total Docs. (3years),Total Refs.,Total Cites (3years),Citable Docs. (3years),Cites / Doc. (2years),Ref. / Doc.,Country,Region,Publisher,Coverage,Categories,Areas
i64,i64,str,str,str,str,str,i64,i64,i64,i64,i64,i64,str,str,str,str,str,str,str,str
1,28773,"""Ca-A Cancer Jo...","""journal""","""15424863, 0007...","""86,091""","""Q1""",198,44,118,4268,30318,85,"""299,99""","""97,00""","""United States""","""Northern Ameri...","""Wiley-Blackwel...","""1950-2022""","""Hematology (Q1...","""Medicine"""
2,29431,"""Quarterly Jour...","""journal""","""00335533, 1531...","""36,730""","""Q1""",292,36,122,2398,2141,122,"""14,83""","""66,61""","""United Kingdom...","""Western Europe...","""Oxford Univers...","""1886-2022""","""Economics and ...","""Economics, Eco..."
3,20315,"""Nature Reviews...","""journal""","""14710072, 1471...","""34,201""","""Q1""",485,121,328,10804,13331,156,"""35,47""","""89,29""","""United Kingdom...","""Western Europe...","""Nature Publish...","""2000-2022""","""Cell Biology (...","""Biochemistry, ..."
4,18434,"""Cell""","""journal""","""00928674, 1097...","""26,494""","""Q1""",856,420,1637,27609,67791,1440,"""43,80""","""65,74""","""United States""","""Northern Ameri...","""Cell Press""","""1974-2022""","""Biochemistry, ...","""Biochemistry, ..."
5,15847,"""New England Jo...","""journal""","""00284793, 1533...","""26,015""","""Q1""",1130,1410,4561,14396,133956,1854,"""33,93""","""10,21""","""United States""","""Northern Ameri...","""Massachussetts...","""1945-2022""","""Medicine (misc...","""Medicine"""


In [12]:
fz.fuzz.partial_ratio('John Lawrence Adgate', 'John Adgate')

73

In [13]:
fz.fuzz.partial_ratio('John Lawrence Adgate', 'John A. Aalborg')

47

In [185]:
x = x.with_columns(
    pl.concat_str([
        pl.col('First Name'),
        pl.col('Middle Initial').fill_null(' '),
        pl.col('Last Name')
    ], sep=" ").alias('Full Name')
)

In [186]:
import httpx
from ratelimit import limits, sleep_and_retry


@sleep_and_retry
@limits(calls=10, period=1)
async def get_orcid_by_name_and_institution(session, first_name: str, last_name: str, institution_name: str) -> str:

    # Construct the query URL
    url = f"https://pub.orcid.org/v3.0/search/?q=given-names:{first_name}%20AND%20family-name:{last_name}%20AND%20affiliation-org-name:{institution_name}"

    # Send a GET request to the ORCID API
    response = await session.get(url, headers={"Accept": "application/json"})

    # Check for a successful response
    if response.status_code == 200:
        data = response.json()
        # Process the search results in the 'data' variable
        if data.get('num-found')==1:
            return data.get('result')[0].get('orcid-identifier').get('path')
    else:
        return None

In [187]:
res = []
async def get_orcids():
    async with httpx.AsyncClient() as client:
        coros = []

        for i in x.iter_rows(named=True):
            coros.append(get_orcid_by_name_and_institution(client,i['First Name'],i['Last Name'],'colorado'))
        
        res = await asyncio.gather(*coros)
        return(res)
    
res2 = await get_orcids()

In [189]:
print(x.with_columns(
    new_orcid = pl.Series(res2)
).select(['ORCID','Full Name','new_orcid']))

shape: (490, 3)
┌────────────────────────────┬──────────────────┬─────────────────────┐
│ ORCID                      ┆ Full Name        ┆ new_orcid           │
│ ---                        ┆ ---              ┆ ---                 │
│ str                        ┆ str              ┆ str                 │
╞════════════════════════════╪══════════════════╪═════════════════════╡
│ null                       ┆ John   Adgate    ┆ 0000-0003-1271-3465 │
│ null                       ┆ Chapla   Agarwal ┆ null                │
│ Orcid: 0000-0002-9455-0235 ┆ Rajesh   Agarwal ┆ null                │
│ Orcid: 0000-0002-2690-2630 ┆ Natalie G Ahn    ┆ 0000-0002-2690-2630 │
│ ...                        ┆ ...              ┆ ...                 │
│ Orcid: 0000-0003-0045-2974 ┆ Rui   Zhao       ┆ null                │
│ Orcid: 0000-0001-5324-6264 ┆ Hongjin   Zheng  ┆ null                │
│ Orcid: 0000-0001-7514-3522 ┆ Shuo   Zhou      ┆ null                │
│ Orcid: 0000-0003-1434-6590 ┆ Yuwen   Zhu      

In [194]:
x.with_columns(
    new_orcid= pl.Series(res2)
).head()


Member ID,Last Name,First Name,Middle Initial,Program,Type,Status,Status Date,Institution,School,Dept,Div,Email,ORCID,Full Name,new_orcid
i64,str,str,str,str,str,str,str,str,str,str,str,str,str,str,str
2004,"""Adgate""","""John""",null,"""CPC""","""Full""","""Active""","""10-Apr-20""","""UCD""","""School of Publ...","""Environmental ...",null,"""john.adgate@cu...",null,"""John Adgate""","""0000-0003-1271..."
1392,"""Agarwal""","""Chapla""",null,"""CPC""","""Affiliate""","""Active""","""29-Nov-03""","""UCD""","""Skaggs School ...","""Pharmaceutical...",null,"""chapla.agarwal...",null,"""Chapla Agarw...",null
1100,"""Agarwal""","""Rajesh""",null,"""CPC""","""Full""","""Active""","""01-Sep-98""","""UCD""","""Skaggs School ...","""Pharmaceutical...",null,"""rajesh.agarwal...","""Orcid: 0000-00...","""Rajesh Agarw...",null
1260,"""Ahn""","""Natalie""","""G""","""MCO""","""Full""","""Active""","""01-Aug-94""","""CU-Boulder""","""College of Art...","""Chemistry & Bi...",null,"""Natalie.Ahn@co...","""Orcid: 0000-00...","""Natalie G Ahn""","""0000-0002-2690..."
2091,"""Ahrendt""","""Gretchen""","""McGuire""","""CPC""","""Clinical""","""Active""","""09-Sep-22""","""UCD""","""School of Medi...","""Surgery""","""Surgical Oncol...","""gretchen.ahren...",null,"""Gretchen McGui...",null


In [195]:
import httpx

# Replace with the ORCID iD you want to retrieve works for
orcid_id = "0000-0002-8991-6458"  # Replace with the desired ORCID iD

# Construct the API endpoint URL for the works associated with the ORCID iD
url = f"https://pub.orcid.org/v3.0/{orcid_id}/works"

ids = []
titles = []

async def get_orcid_works():
    async with httpx.AsyncClient() as client:
        response = await client.get(url, headers={"Accept": "application/json"})

        # Check for a successful response
        if response.status_code == 200:
            data = response.json()
            
            # The 'data' variable contains the works associated with the specified ORCID iD
            # You can now process and extract the information you need from the 'data' variable
            for work in data.get("group"):
                print("Title:", work.get("work-summary")[0].get("title").get("title").get('value'))
                titles.append(work.get("work-summary")[0].get("title").get("title").get('value'))
                #print("Publication Date:", work.get("publication-date"))
                #print("Doi:", work.get("external-ids").get("external-id")[0].get("external-id-value"))
                print(work.get("external-ids").get("external-id")[0].get("external-id-value"))
                ids.append(work.get("external-ids").get("external-id")[0].get("external-id-value"))
        else:
            print(f"Error: {response.status_code}")

# Run the asynchronous function
await get_orcid_works()

Title: Integration of 168,000 samples reveals global patterns of the human gut microbiome
10.1101/2023.10.11.560955
Title: bamSliceR: cross-cohort variant and allelic bias analysis for rare variants and rare diseases
10.1101/2023.09.15.558026
Title: GenomicSuperSignature facilitates interpretation of RNA-seq experiments through robust, efficient comparison to public databases
10.1038/s41467-022-31411-3
Title: HGNChelper: identification and correction of invalid gene symbols for human and mouse [version 2; peer review: 3 approved]
10.12688/f1000research.28033.2
Title: Ten simple rules for large-scale data processing
10.1371/journal.pcbi.1009757
Title: GenomicSuperSignature: interpretation of RNA-seq experiments through robust, efficient comparison to public databases
10.1101/2021.05.26.445900
Title: HGNChelper: identification and correction of invalid gene symbols for human and mouse
10.12688/f1000research.28033.1
Title: HGNChelper: identification and correction of invalid gene symbols 

In [196]:
%pip install ratelimit

from ratelimit import limits, sleep_and_retry

@sleep_and_retry
@limits(calls=20, period=1)
async def pubmed_record_by_title(session, title: str):
    url = f'https://www.ebi.ac.uk/europepmc/webservices/rest/search?query=title:"{title}" AND author:Davis&format=json&resultType=core'
    resp = await session.get(url, timeout=30)
    pmids=[]
    for res in resp.json().get('resultList').get('result'):
        try:
            pmids.append(res['pmid'])
        except KeyError:
            pass
    print(len(pmids))
    if len(pmids)==1:
        return pmids[0]
    else:
        return None



async def main(titles):
    async with httpx.AsyncClient() as session:  #use httpx
        res = await asyncio.gather(*[pubmed_record_by_title(session, x) for x in titles])
        res = list(zip(titles,res))
        print(res)
        
await main(titles)


[notice] A new release of pip is available: 23.1.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.
0
0
1
0
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
0
1
1
0
1
1
1
0
1
1
0
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
1
1
[('Integration of 168,000 samples reveals global patterns of the human gut microbiome', None), ('bamSliceR: cross-cohort variant and allelic bias analysis for rare variants and rare diseases', None), ('GenomicSuperSignature facilitates interpretation of RNA-seq experiments through robust, efficient comparison to public databases', '35760813'), ('HGNChelper: identification and correction of invalid gene symbols for human and mouse [version 2; peer review: 3 approved]', '33564398'), ('Ten simple rules for large-scale data processing', '35143491'), ('GenomicSuperSignature: interpretation of RNA-seq experiments throug

In [197]:
pmc_ids = pl.read_csv('/Users/seandavis/Downloads/PMC-ids.csv.gz',infer_schema_length=0)

In [198]:
pmc_ids.filter(pl.col('DOI').is_in(ids))

Journal Title,ISSN,eISSN,Year,Volume,Issue,Page,DOI,PMCID,PMID,Manuscript Id,Release Date
str,str,str,str,str,str,str,str,str,str,str,str
"""Genome Res""","""1088-9051""","""1549-5469""","""2006""","""16""","""1""","""123""","""10.1101/gr.407...","""PMC1356136""","""16344561""",null,"""live"""
"""PLoS Genet""","""1553-7390""","""1553-7404""","""2006""","""2""","""4""","""e51""","""10.1371/journa...","""PMC1428788""","""16604156""",null,"""live"""
"""Am J Hum Genet...","""0002-9297""","""1537-6605""","""1997""","""61""","""6""","""1431""","""10.1086/301635...","""PMC1716077""","""9399893""",null,"""live"""
"""Nature""","""0028-0836""","""1476-4687""","""2007""",null,null,"""799""","""10.1038/nature...","""PMC2212820""","""17571346""","""NIHMS27513""","""live"""
"""Proc Natl Acad...","""0027-8424""","""1091-6490""","""2008""","""105""","""5""","""1674""","""10.1073/pnas.0...","""PMC2234203""","""18227516""",null,"""live"""
"""Breast Cancer ...","""0167-6806""","""1573-7217""","""2008""","""113""","""2""","""217""","""10.1007/s10549...","""PMC2615075""","""18266105""","""NIHMS42188""","""live"""
"""Endocrinology""","""0013-7227""","""1945-7170""","""2009""","""150""","""4""","""1766""","""10.1210/en.200...","""PMC2659280""","""19131569""",null,"""live"""
"""PLoS One""",null,"""1932-6203""","""2009""","""4""","""4""","""e5415""","""10.1371/journa...","""PMC2671847""","""19404404""",null,"""live"""
"""Mol Cancer Res...","""1541-7786""","""1557-3125""","""2009""","""7""","""9""","""1438""","""10.1158/1541-7...","""PMC2746883""","""19723875""","""NIHMS134488""","""live"""


In [199]:
from cu_profile_utils.profile_page import ProfilePage

In [205]:
p = ProfilePage('9687211') # Manali Kamdar

In [206]:
res = await p.get_pmids_as_list()

In [207]:
len(res)

54

In [208]:
res2 = await p.get_mesh_term_similarities_as_dataframe()

In [209]:
res2

,profile_id,mesh_term,publication_count,most_recent_publication_year,publication_count_all_authors,concept_score
0,9687211,"Lymphoma, Large B-Cell, Diffuse",14,2023,89,5.63
1,9687211,"Leukemia, Lymphocytic, Chronic, B-Cell",9,2023,58,3.03
2,9687211,"Lymphoma, Mantle-Cell",6,2023,14,2.94
3,9687211,"Lymphoma, B-Cell",5,2022,87,2.48
4,9687211,Hematopoietic Stem Cell Transplantation,9,2023,538,2.47
...,...,...,...,...,...,...
187,9687211,Single-Blind Method,1,1993,275,0.01
188,9687211,Family Practice,1,1997,486,0.01
189,9687211,"Administration, Oral",1,1993,790,0.01
190,9687211,Acute Disease,1,1993,986,0.01
